In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *

In [ ]:
positives = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input/table_data_input.csv')
negatives = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input/negative_table_data_input.csv')
others = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input/ambiguous_table_data_input.csv')

In [ ]:
from shapely import wkt

positives.drop(index=positives.index[0], inplace=True)
negatives.drop(index=negatives.index[0], inplace=True)
others.drop(index=others.index[0], inplace=True)

positives['ignition_point (epsg=3857)'] = positives['ignition_point (epsg=3857)'].apply(wkt.loads)
positives = gpd.GeoDataFrame(positives, geometry='ignition_point (epsg=3857)', crs='EPSG:3857')
negatives['ignition_point (epsg=3857)'] = negatives['ignition_point (epsg=3857)'].apply(wkt.loads)
negatives = gpd.GeoDataFrame(negatives, geometry='ignition_point (epsg=3857)', crs='EPSG:3857')
others['ignition_point (epsg=3857)'] = others['ignition_point (epsg=3857)'].apply(wkt.loads)
others = gpd.GeoDataFrame(others, geometry='ignition_point (epsg=3857)', crs='EPSG:3857')

In [ ]:
for c in positives.columns:
    if c not in negatives.columns:
        print(f'Column {c} in positives not in negatives')
    if c not in others.columns:
        print(f'Column {c} in positives not in others')

for c in negatives.columns:
    if c not in positives.columns:
        print(f'Column {c} in negatives not in positives')
    if c not in others.columns:
        print(f'Column {c} in negatives not in others')

for c in others.columns:
    if c not in positives.columns:
        print(f'Column {c} in others not in positives')
    if c not in negatives.columns:
        print(f'Column {c} in others not in negatives')

In [ ]:
geonegatives = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/negatives.geojson').to_crs(epsg=3857)
geonegatives['ignition_point (epsg=3857)'] = geonegatives.geometry.centroid
geoothers = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/ambiguous.geojson').to_crs(epsg=3857)
geoothers['ignition_point (epsg=3857)'] = geoothers.geometry.centroid

In [ ]:
from shapely.geometry import Point

def round_points(gdf, col, precision=3):
    gdf = gdf.copy()
    gdf[col] = gdf[col].apply(
        lambda geom: Point(round(geom.x, precision), round(geom.y, precision))
        if geom.geom_type == 'Point' else geom
    )
    return gdf

negatives = round_points(negatives, 'ignition_point (epsg=3857)')
others = round_points(others, 'ignition_point (epsg=3857)')

In [ ]:
gdf1 = negatives.copy()
gdf1['x'] = gdf1['ignition_point (epsg=3857)'].x
gdf1['y'] = gdf1['ignition_point (epsg=3857)'].y

gdf2 = geonegatives.copy()
gdf2['x'] = gdf2['ignition_point (epsg=3857)'].x
gdf2['y'] = gdf2['ignition_point (epsg=3857)'].y

gdf2['x'] = gdf2.apply(lambda row: round(float(np.format_float_positional(row.x, trim='-')), 3), axis=1)
gdf2['y'] = gdf2.apply(lambda row: round(float(np.format_float_positional(row.y, trim='-')), 3), axis=1)

merged = gdf1.merge(gdf2[['id', 'x', 'y']], on=['x','y'], how='inner')

# Restore geometry
merged = gpd.GeoDataFrame(merged, geometry='ignition_point (epsg=3857)', crs=3857).drop(columns=['x', 'y']).rename(columns={'id': 'fire_id'})

In [ ]:
save_clean_data(merged, 'negative_model_data_input', '/nfs/home/genovese/thesis-wildfire-genovese/database/cache')

---

In [ ]:
gdf1 = others.copy()
gdf1['x'] = gdf1['ignition_point (epsg=3857)'].x
gdf1['y'] = gdf1['ignition_point (epsg=3857)'].y

gdf2 = geoothers.copy()
gdf2['x'] = gdf2['ignition_point (epsg=3857)'].x
gdf2['y'] = gdf2['ignition_point (epsg=3857)'].y

gdf2['x'] = gdf2.apply(lambda row: round(float(np.format_float_positional(row.x, trim='-')), 3), axis=1)
gdf2['y'] = gdf2.apply(lambda row: round(float(np.format_float_positional(row.y, trim='-')), 3), axis=1)

merged = gdf1.merge(gdf2[['id', 'x', 'y']], on=['x','y'], how='inner')

# Restore geometry
merged = gpd.GeoDataFrame(merged, geometry='ignition_point (epsg=3857)', crs=3857).drop(columns=['x', 'y']).drop(columns='fire_id').rename(columns={'id': 'fire_id'})

In [ ]:
save_clean_data(merged, 'others_model_data_input', '/nfs/home/genovese/thesis-wildfire-genovese/database/cache')

---

In [ ]:
geopositives = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont_2016_2024_fa.geojson')

In [ ]:
geopositives['ignition_point (epsg=3857)'] = geopositives.apply(lambda row: Point(row.point_x, row.point_y), axis=1)

In [ ]:
gdf1 = positives.copy()
gdf1['x'] = gdf1['ignition_point (epsg=3857)'].x
gdf1['y'] = gdf1['ignition_point (epsg=3857)'].y

gdf2 = geopositives.copy()
gdf2['x'] = gdf2['ignition_point (epsg=3857)'].x
gdf2['y'] = gdf2['ignition_point (epsg=3857)'].y

gdf2['x'] = gdf2.apply(lambda row: round(float(np.format_float_positional(row.x, trim='-')), 3), axis=1)
gdf2['y'] = gdf2.apply(lambda row: round(float(np.format_float_positional(row.y, trim='-')), 3), axis=1)

gdf1['x'] = gdf1.apply(lambda row: round(float(np.format_float_positional(row.x, trim='-')), 3), axis=1)
gdf1['y'] = gdf1.apply(lambda row: round(float(np.format_float_positional(row.y, trim='-')), 3), axis=1)

merged = gdf1.merge(gdf2[['id', 'x', 'y']], on=['x','y'], how='inner')

# Restore geometry
merged = gpd.GeoDataFrame(merged, geometry='ignition_point (epsg=3857)', crs=3857).drop(columns=['x', 'y']).drop(columns='fire_id').rename(columns={'id': 'fire_id'})

In [ ]:
save_clean_data(merged, 'positive_model_data_input', '/nfs/home/genovese/thesis-wildfire-genovese/database/cache')

---

In [ ]:
positives = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/database/cache/positive_model_data_input.geojson')
negatives = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/database/cache/negative_model_data_input.geojson')
others = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/database/cache/others_model_data_input.geojson')

In [ ]:
total = pd.concat([positives, negatives, others], axis=0, ignore_index=True)

In [ ]:
total.duplicated().sum()

In [ ]:
save_clean_data(total, 'dataset', '/nfs/home/genovese/thesis-wildfire-genovese/database/model_input')

---